### start
`run --net=test -v $HOME/git:/opt/git -e PYTHONPATH=/opt/git/aiomysql -p 8888:8888 -w /opt/git/aiomysql jupyter/minimal-notebook`
### notes
* this notebook already has an event loop running, so `await` just works
* run the notebook on the same docker network (`--net`) as the `mysql` container

In [1]:
from aiomysql.connection import MysqlConnection

In [2]:
help(MysqlConnection.connect)

Help on method connect in module aiomysql.connection:

async connect(host, user, password='', database='', port=3306, autocommit=False, isolation=None) method of builtins.type instance
    connect and authenticate to mysql
    
    host       - hostname or IP address of mysql server
    user       - mysql user
    password   - password for mysql user
    database   - name of database
    port       - mysql listening port
    autocommit - autocommit for connection (else per mysql)
    isolation  - isolation level for connection (else per mysql)
                 (eg: "REPEATABLE READ", "READ UNCOMMITED" ...)
    
    Return:
        aiomysql.connection.MysqlConnection instance
    
    Notes:
        * the connection remains open until closed by the mysql server,
          or until the close method is called



In [3]:
con = await MysqlConnection.connect("mysql", "test")

In [4]:
columns, rows = await con.execute("select now() as time")

In [5]:
columns

['time']

In [6]:
rows

[[datetime.datetime(2021, 1, 6, 21, 24, 1)]]

In [7]:
help(con.execute)

Help on method execute in module aiomysql.connection:

async execute(query) method of aiomysql.connection.MysqlConnection instance
    execute a SQL command and return any result
    
    query - command to be executed
    
    Return:
        [column names], [[row], ...]
    
    Notes:
        * for non-SELECT commands the return will be two empty lists
        * if an INSERT with AUTOINCREMENT occurs, the autoincrement
          value is available by calling the last_id method
        * if the command generates a message, the message is available
          by calling the last_message method



In [8]:
await con.execute("use bob")
await con.execute("desc person")

(['Field', 'Type', 'Null', 'Key', 'Default', 'Extra'],
 [['id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'],
  ['create_time', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', ''],
  ['update_time',
   'timestamp',
   'NO',
   '',
   'CURRENT_TIMESTAMP',
   'on update CURRENT_TIMESTAMP'],
  ['name', 'varchar(50)', 'NO', 'UNI', None, ''],
  ['email', 'varchar(50)', 'YES', 'UNI', None, '']])

In [9]:
await con.execute("INSERT INTO person (name) VALUES ('bob')")

([], [])

In [10]:
con.last_id()

4

In [11]:
con.last_message()

''

In [12]:
await con.execute("select * from person")

(['id', 'create_time', 'update_time', 'name', 'email'],
 [[4,
   datetime.datetime(2021, 1, 6, 21, 24, 1),
   datetime.datetime(2021, 1, 6, 21, 24, 1),
   'bob',
   None]])